CHARNAY Paul - MONEDIERES Emmeline

# Transformée en Ondelettes 2D, application au traitement des images

In [ ]:
# Librairies de base
import numpy as np
import pandas as pd
import pywt
import scipy as scp
import scipy.io as sio
import itertools

# Affichage
import pylab as pyl
import matplotlib.pyplot as plt
import holoviews as hv
import param
import panel as pn
import hvplot.pandas
from bokeh.models import HoverTool
from panel.pane import LaTeX
hv.extension('bokeh')
%matplotlib inline

## Approximation linéaire et non linéaire.

In [ ]:
Blocks2, Piece, im, im2, Mi = np.load('dataTP.npy', allow_pickle=True)
imagesRef = {"Lenna": im2, "Canaletto": im}
imagesRef_coul = {"Minotaure": np.clip(Mi, 0, 255).astype('uint8'), "Cartoon": plt.imread("Cartoon.jpg"), "Cartoon_bruit": plt.imread("Cartoon_bruitee.jpeg")}
options = dict(cmap='gray', xaxis=None, yaxis=None, width=400, height=400, toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options), hv.Raster(imagesRef["Canaletto"]).opts(**options))

In [ ]:
size = 400
WT = pywt.wavedecn(im, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray', width=size, height=size)

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def ApproxOnd2D(S, qmf, L, threshold):
    WTB = pywt.wavedecn(S, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WTB)
    WTS = arr * (np.abs(arr) > threshold)
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Srec = pywt.waverecn(coeffs_from_arr, qmf, mode='per')
    ncoeffs = np.shape(WTS[WTS != 0])[0]
    return Srec, ncoeffs

Ecrire une focntion PSNR.

In [ ]:
def PSNR(I, Iref):
    mse = np.mean((Iref - I) ** 2)
    if mse == 0:
        return 100
    Val_MAX = np.max(Iref)
    return 20 * np.log10(Val_MAX / np.sqrt(mse))

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2nonlin(I, qmf, L, N):
    WTB = pywt.wavedecn(I, qmf, mode='per', level=L)
    arr, coeff_slices = pywt.coeffs_to_array(WTB)
    WTS = arr * (np.abs(arr) > np.sort(np.abs(arr), axis=None)[-N])
    coeffs_from_arr = pywt.array_to_coeffs(WTS, coeff_slices)
    Irec = pywt.waverecn(coeffs_from_arr, qmf, mode='per')
    psnr = PSNR(I, Irec)
    Irec = np.clip(Irec, 0, 255)
    return Irec, psnr

In [ ]:
Irec, psnr = ApproxOnd2nonlin(im,'db2',6,5000)
hv.Image(Irec).opts(cmap='gray',width=400,height=400)

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
class Approx2D(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto", objects=imagesRef.keys())
    wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
    wave = param.ObjectSelector(default="haar", objects=wavelist)
    L = param.Integer(7, bounds=(0,7))
    N = param.Integer(2000, bounds=(1,10000))
    
    @param.depends('wave', 'N', 'L')
    def view(self):
        Im = imagesRef[self.image]
        Irec, psnr = ApproxOnd2nonlin(Im, self.wave, self.L, self.N)
        strp1 = "%2.2f" % psnr
        te1 = 'PSNR image bruitée'
        TN = pn.Column(LaTeX(te1, size=15, dpi=100), LaTeX(strp1, size=15, dpi=100))
        return pn.Row(hv.Image(Irec).opts(cmap='gray', width=400, height=400), TN)

In [ ]:
approx2D = Approx2D()
pn.Row(approx2D.param, approx2D.view)

On observe que plus on garde de coefficients plus le PSNR est élevé, c'est logique car on s'éloigne moins de l'image originale.
Avec l'ondelette de Haar, on voit apparaître des carrés uniformes. C'est normal car cette ondelette est constante par morceaux, alors qu'une photographie ne l'est pas en général (il y a des nuances de couleur). En revanche si on prenait l'image Cartoon, Haar serait bien adaptée.

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2nonlin

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image': imagesRef, 'L': np.linspace(2, 7, 6, dtype=int), 'N': np.linspace(800, 10000, 30, dtype=int),'wave': wavelist}
dfexp = pd.DataFrame(list(itertools.product(*experiences.values())), columns=experiences.keys())

In [ ]:
print(dfexp)

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:
def row2PSNR(row):
    p = ApproxOnd2nonlin(imagesRef[row.Image], row.wave, row.L, row.N)[1]
    return {'PSNR':p}

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
dfexp[['PSNR']] = pd.DataFrame.from_records(dfexp.apply(row2PSNR,axis=1).values)

In [ ]:
print(dfexp)

Utiliser hvplot pour visualiser la base de données.

In [ ]:
h = HoverTool()

In [ ]:
dfexp.hvplot('L', 'PSNR', by='wave', kind='scatter', groupby=['Image', 'N']).opts(width=600,tools = [h]).redim.range(PSNR=(0, 50), L=(1, 8))

Il semblerait judicieux d'éliminer l'ondelette de Haar qui s'avère sous performante, quelles que soient les valeurs de L et de N, pour les deux images étudiées. Cette remarque semble normale car une image n'est pas forcément constante par morceaux.

De plus, le PSNR est meilleur avec les décompositions les plus profondes.

## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
def SeuillageDurOndelettes(I, qmf, L, Seuil):
    WTB = pywt.wavedecn(I, qmf, mode='per', level=L)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WTB)
    WTS = arr * (np.abs(arr) > Seuil)
    coeffs_from_arr = pywt.unravel_coeffs(WTS, coeff_slices, coeff_shapes)
    Irec = pywt.waverecn(coeffs_from_arr, qmf, mode='per')
    Irec = np.clip(Irec, 0, 255)
    psnr = PSNR(Irec, I)
    return Irec, psnr

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
class WaveSeuillage(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(10,bounds=(1,1000))
    @param.depends('wave', 'Seuil', 'L')
    def view(self):
        Im = imagesRef[self.image]
        Irec, psnr = SeuillageDurOndelettes(Im, self.wave, self.L, self.Seuil)
        strp1 = "%2.2f" % psnr
        te1 = 'PSNR image seuillée'
        TN = pn.Column(LaTeX(te1,size=15,dpi=100), LaTeX(strp1,size=15,dpi=100))
        return pn.Row(hv.Image(Irec).opts(cmap='gray', width=400, height=400), TN)

In [ ]:
waveSeuillage = WaveSeuillage()
pn.Row(waveSeuillage.param, waveSeuillage.view)

In [ ]:
n1, n2 = np.shape(im)
B = np.random.randn(n1, n2)
sigma = 10
ib = im + sigma * B
ib = np.clip(ib, 0, 255)
hv.Image(ib).opts(cmap='gray', width=400, height=400)

Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitéeset débruitées.

In [ ]:
def im_bruit(Im, sigma, seed):
    n1, n2 = np.shape(Im)
    np.random.seed(seed=seed)
    B = np.random.randn(n1, n2)
    ib = Im + sigma * B
    ib = np.clip(ib, 0, 255)
    return ib

In [ ]:
class WaveDebruit(param.Parameterized):
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="db3",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(2.6,bounds=(0, 20))
    Sigma = param.Number(10,bounds=(1, 30))
    seednoise = param.Integer(1,bounds=(0,50))
    @param.depends('wave', 'Seuil', 'L')
    def view(self):
        Im = imagesRef[self.image]
        image_bruit = im_bruit(Im, self.Sigma, self.seednoise)
        psnr_bruit = PSNR(image_bruit, Im)
        image_debruit = SeuillageDurOndelettes(image_bruit, self.wave, self.L, self.Seuil * self.Sigma)[0]
        psnr_deb = PSNR(image_debruit, Im)
        strp1 = "%2.2f" % psnr_bruit
        te1 = 'PSNR image bruitée'
        strp2 = "%2.2f" % psnr_deb
        te2 = 'PSNR image débruitée'
        TN = pn.Column(LaTeX(te1,size=15,dpi=100), LaTeX(strp1,size=15,dpi=100), LaTeX(te2,size=15,dpi=100), LaTeX(strp2,size=15,dpi=100))
        return pn.Row(pn.Column(hv.Image(Im).opts(cmap='gray', width=400, height=400, title="Image originale"), hv.Image(image_bruit).opts(cmap='gray', width=400, height=400, title="Image bruitée")), pn.Column(hv.Image(image_debruit).opts(cmap='gray', width=400, height=400, title="Image débruitée"), TN))

In [ ]:
wavedebruit = WaveDebruit()
pn.Column(wavedebruit.param, wavedebruit.view)

On observe que le débruitage est efficace, mais que le résultat n'est tout de même pas très net.

## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTranslation(IB, wave, seuil, NbT, I):
    N1, N2 = np.shape(IB)
    Lmax = pywt.dwtn_max_level((N1, N2), pywt.Wavelet(wave))
    ISum = np.zeros((N1, N2))
    P = []
    for j, k in itertools.product(range(NbT), range(NbT)):
        IBtemp = np.roll(np.roll(IB, j, axis=0), k, axis=1) 
        Irectemp = SeuillageDurOndelettes(IBtemp, wave, Lmax, seuil)[0]
        Irectemp2 = np.roll(np.roll(Irectemp, -k, axis=1), -j, axis=0)
        ISum = ISum + Irectemp2
        Irec = ISum / (len(P) + 1)
        P.append(PSNR(I, Irec))
    return Irec, np.array(P)

Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

In [ ]:
class Debruit_translat(param.Parameterized):
    image = param.ObjectSelector(default="Lenna", objects=imagesRef.keys())
    wave = param.ObjectSelector(default="db3", objects=wavelist)
    Sigma = param.Number(10, bounds=(1, 30))
    seednoise = param.Integer(1, bounds=(0, 50))
    L = param.Integer(7,bounds=(1, 7))
    Seuil = param.Number(2.6, bounds=(0, 4))
    NbT = param.Integer(7, bounds=(1, 15))
    @param.depends('wave', 'Seuil', 'L', 'NbT', 'Sigma', 'seednoise')
    def view(self):
        Im = imagesRef[self.image]
        image_bruit = im_bruit(Im, self.Sigma, self.seednoise)
        psnr_bruit = PSNR(image_bruit, Im)
        image_debruit, P_deb = DebruitTranslation(image_bruit, self.wave, self.Seuil * self.Sigma, self.NbT, Im)
        strp1 = "%2.2f" % psnr_bruit
        te1 = 'PSNR image bruitée'
        strp2 = "%2.2f" % P_deb[-1]
        te2 = 'PSNR image débruitée'
        TN = pn.Column(LaTeX(te1,size=15,dpi=100), LaTeX(strp1,size=15,dpi=100), LaTeX(te2,size=15,dpi=100), LaTeX(strp2,size=15,dpi=100), LaTeX("NbT",size=15,dpi=100), LaTeX(str(self.NbT),size=15,dpi=100))
        return pn.Row(pn.Column(hv.Image(Im).opts(cmap='gray', width=400, height=400, title="Image originale"), hv.Image(image_bruit).opts(cmap='gray', width=400, height=400, title="Image bruitée")), pn.Column(hv.Image(image_debruit).opts(cmap='gray', width=400, height=400, title="Image débruitée"), TN))

In [ ]:
wavedebruit_translat = Debruit_translat()
pn.Column(wavedebruit_translat.param, wavedebruit_translat.view)

Avec les translations (7 par défaut) et pour les mêmes paramètres que le débruitage sans translation, on observe un gain de PSNR de 3 points, et l'image débruitée est beaucoup plus nette.

### Plan d'expériences pour évaluer l'impact des translations  

Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

On choisit de ne pas tester toutes les ondelettes pour avoir un temps de calcul raisonnable, et car les résultats sont de toute façon très similaires.

In [ ]:
wavelist = ['haar', 'db3', 'db4', 'coif3']
experiences_DebruitTrans = {'Image': imagesRef, 'wave': wavelist, "Seuil": np.linspace(1, 4, 6)}
dfexp_DebruitTrans = pd.DataFrame(list(itertools.product(*experiences_DebruitTrans.values())), columns=experiences_DebruitTrans.keys())

In [ ]:
print(dfexp_DebruitTrans)

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
NbT_max = 5
sigma = 10
def Debruit_Translat_PSNRMoyen(Im, wave, Seuil, n=4):
    return np.mean([DebruitTranslation(im_bruit(Im, sigma, seed), wave, Seuil, NbT_max, Im)[1] for seed in range(n)], axis=0)  # Sigma fixé comme un barbare

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row):
    return {'PSNR': Debruit_Translat_PSNRMoyen(imagesRef[row.Image], row.wave, row.Seuil * sigma, NbT_max)}

In [ ]:
dfexp_DebruitTrans[['PSNR']] = pd.DataFrame.from_records(dfexp_DebruitTrans.apply(row2DebruitTrans, axis=1).values)

In [ ]:
df = dfexp_DebruitTrans.copy()
df = pd.concat((df[['Image', 'wave', "Seuil"]],pd.DataFrame(df.PSNR.values.tolist(),df.index)),axis=1)
df = df.melt(id_vars=['Image', 'wave', "Seuil"], var_name='NbT', value_name='PSNR')
print(df)

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

In [ ]:
h = HoverTool()
df.hvplot('NbT', 'PSNR', by='wave', kind='scatter', groupby=['Image', 'Seuil']).opts(width=600,tools = [h]).redim.range(PSNR=(25, 37), NbT=(0, 25))

Quel que soit le seuil, on voit bien que les translations permettent d'améliorer le PSNR. Entre 0 et 10 translations, on observe un fort gain de PSNR, après 10 translations, il augmente beaucoup moins vite.

De plus, on voit bien qu'en augmentant le seuil jusqu'à un certain point, le PSNR augmente et atteint son maximum. Mais si on continue d'augmenter le seuil, le PSNR diminue à nouveau car on dégrade l'image.

## Débruitage d'une image couleur.

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissances de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vrai problème, il faut juste que les dimensions soient des multiples de puissances de 
2. C'est la puissance de 2 qui définira l'échalle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

Rédiger également une fonction prenant en entrée un nom de fichier 
permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question.
On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

In [ ]:
wavelist = ['haar', 'db2', 'db3', 'db4', 'coif1', 'coif2', 'coif3']

In [ ]:
def EstimEcartTypeBruit(Im, qmf):
    N1, N2 = np.shape(Im)[:2]
    Lmax = pywt.dwtn_max_level((N1, N2), pywt.Wavelet(qmf))
    wsb = pywt.wavedec(Im, qmf, mode='per', level=Lmax)
    mt = np.sqrt(2) * scp.special.erfinv(0.5)
    return np.median(np.abs(wsb[Lmax])[wsb[Lmax] != 0]) / mt

In [ ]:
def Debruit_couleur(Im, wave, seuil, NbT):
    n, m = np.shape(Im)[:2]
    new_n = int(2 ** np.ceil(np.log2(n)))
    new_m = int(2 ** np.ceil(np.log2(m)))
    Im_sym = np.zeros((new_n, new_m, 3))
    sigma = EstimEcartTypeBruit(Im, str(wave))
    for i in range(3):
        Im_sym[:n, :m, i] = Im[:n, :m, i]  # On recopie l'image de base
        Im_sym[n:, :m, i] = Im[:-(new_n - n +1):-1, :m, i]  # on symétrise selon les lignes
        Im_sym[:, m: , i] = Im_sym[:, :-(new_m - m + 1):-1, i]  # on symétrise selon les colonnes
        Im_sym[:, :, i], psnr = DebruitTranslation(Im_sym[:, :, i], wave, sigma * seuil, NbT, Im_sym[:, :, i])
        Im_sym[:, :, i] = np.clip(Im_sym[:, :, i], 0, 255)
    return Im_sym[:n, :m, :].astype('uint8')

In [ ]:
class Test_debruit_couleur(param.Parameterized):
    image = param.ObjectSelector(default="Minotaure", objects=imagesRef_coul.keys())
    wave = param.ObjectSelector(default="db3", objects=wavelist)
    L = param.Integer(7,bounds=(1, 7))
    Seuil = param.Number(2.5, bounds=(0, 5))
    NbT = param.Integer(3, bounds=(1, 8))
    @param.depends('wave', 'Seuil', 'L', 'NbT')
    def view(self):
        Im = imagesRef_coul[self.image]
        image_debruit = Debruit_couleur(Im, self.wave, self.Seuil, self.NbT)
        haut, larg, _ = np.shape(image_debruit)
        echelle = max(haut // 500, larg // 500)
        return pn.Row(hv.RGB(Im).opts(xlabel=None,ylabel=None, width=larg // echelle, height=haut // echelle, title="Image originale"),
                      hv.RGB(image_debruit).opts(xlabel=None,ylabel=None, width=larg // echelle, height=haut // echelle, title="Image débruitée"), self.Seuil)

In [ ]:
wavedebruit_couleur = Test_debruit_couleur()
pn.Column(wavedebruit_couleur.param, wavedebruit_couleur.view)

Par défaut, l'ondelette choisie pour le Minotaure est la Daubechies 3, car la photo du Minotaure n'est pas constante par morceaux.

Cependant, on peut voir sur la photo du Cartoon que Haar est plus efficace que sur les autres images.

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value, counts = np.unique(x, return_counts=True)
    Proba = counts / len(x)
    Ent = - np.sum(np.log2(Proba) * Proba)
    return Ent

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

In [ ]:
def QuantificationOndelettes(Im, qmf, Pas):
    N1, N2 = np.shape(Im)[:2]
    Lmax = pywt.dwtn_max_level((N1, N2), pywt.Wavelet(qmf))
    WTB = pywt.wavedecn(Im, qmf, mode='per', level=Lmax)
    arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WTB)
    arr = np.round(Pas * np.round(arr / Pas), 1)
    coeffs_from_arr = pywt.unravel_coeffs(arr, coeff_slices, coeff_shapes)
    Irec = pywt.waverecn(coeffs_from_arr, qmf, mode='per')
    Irec = np.clip(Irec, 0, 255)
    psnr = PSNR(Im, Irec)
    ent = ShannonEntropy(arr)
    return Irec, psnr, ent

Créer le dashboard asscoié à la focntion précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
class WaveQuant(param.Parameterized):
    wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
    image = param.ObjectSelector(default="Canaletto", objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    Pas = param.Number(30, bounds=(10, 300))
    options = dict(cmap='gray', xaxis=None, yaxis=None, width=400, height=400, toolbar=None)
    @param.depends('image', 'wave', 'Pas')
    def view(self):
        Im = imagesRef[self.image]
        image_quant, p1, ent2 = QuantificationOndelettes(Im, self.wave, self.Pas)
        te1 = "PSNR image quantifiée"
        te2 = "Entropie image originale"
        te3 = "Entropie image compressée"
        N1, N2 = np.shape(Im)[:2]
        Lmax = pywt.dwtn_max_level((N1, N2), pywt.Wavelet(self.wave))
        WTB = pywt.wavedecn(Im, self.wave, mode='per', level=Lmax)
        arr, coeff_slices, coeff_shapes = pywt.ravel_coeffs(WTB)
        ent1 = ShannonEntropy(arr)
        TN = pn.Column(LaTeX(te1,size=15,dpi=100), LaTeX(str(p1), size=15,dpi=100),
                       LaTeX(te2,size=15,dpi=100), LaTeX(str(ent1), size=15,dpi=100), 
                       LaTeX(te3, size=15, dpi=100), LaTeX(str(ent2), size=15, dpi=100))
        return pn.Row(hv.Raster(Im).opts(**options, title="Image originale"),
                      hv.Raster(image_quant).opts(**options, title="Image quantifiée"), 
                      TN)

In [ ]:
Visu_WaveQuant = WaveQuant()
pn.Column(Visu_WaveQuant.param, Visu_WaveQuant.view)

On voit que plus on augmente le pas de quantification, plus l'entropie et le PSNR diminuent. C'est logique car en augmentant le pas de quantification, on diminue la quantité d'information gardée (donc l'entropie), et on dégrade l'image (donc le PSNR).

Créer dun plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences_quant = {'image': imagesRef, 'QS': np.linspace(10, 400, 40), 'wave': wavelist}
dfexp = pd.DataFrame(list(itertools.product(*experiences_quant.values())), columns=experiences_quant.keys())

In [ ]:
print(dfexp)

In [ ]:
def row2DistorsionRate(row):
    Im, p, ent = QuantificationOndelettes(imagesRef[row.image], row.wave, row.QS)
    return {'Entropie': ent, 'PSNR': p}

In [ ]:
dfexp[['Entropie', 'PSNR']] = pd.DataFrame.from_records(dfexp.apply(row2DistorsionRate,axis=1).values)

In [ ]:
print(dfexp)

In [ ]:
h = HoverTool()
dfexp.hvplot('Entropie', 'PSNR', by='wave', kind='scatter', groupby=['image', 'QS']).opts(width=600,tools = [h]).redim.range(PSNR=(15, 42), Entropie=(0, 4))

In [ ]:
h = HoverTool()
dfexp.hvplot('QS', 'PSNR', by='wave', kind='scatter', groupby=['image'], title="Evolution du PSNR en fonction de la quantification").opts(width=600,tools = [h]).redim.range(PSNR=(15, 42))

In [ ]:
h = HoverTool()
dfexp.hvplot('QS', 'Entropie', by='wave', kind='scatter', groupby=['image'], title="Evolution de l'entropie en fonction de la quantification").opts(width=600,tools = [h]).redim.range(Entropie=(0, 3))

On voit bien sur ces graphiques la diminution du PSNR et de l'entropie en fonction du pas de quantification, comme énoncé précédemment.

## Pour aller plus loin (à titre informatif et optionnel)

Nous proposons ici d'effectuer la compression sur les 3 canau RGB. Or l'oeil humain est plus sensible à la luminance qu'aux composantes purement chromatiques. C'est pourquoi, la plupart des algorithmes de compressions sont effectué dans un espace colorimétrique YUV où Y est la luminance. On alloue alors plus d'information au canal Y et on comprime plus drastiquement les deux autres canaux. Une méthode standart consiste par exemple à sous-échantionner d'un facteur 2 les deux composantes U et V avant de les comprimer. 

https://fr.wikipedia.org/wiki/Sous-échantillonnage_de_la_chrominance

On obtient alors des images de chrominances moins résolues et donc moins lourdes mais le rendu final reste correct car l'oeil humain est nettement plus sensible à la luminance. 